In [1]:
import os
import winreg
import win32com.client
import pandas as pd
import re
from datetime import date, datetime, timedelta

 # 바로가기 파일경로를 사용하여 대상 파일의 실제 경로 찾아 반환
def get_target_path(lnk_path):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_path)
    target_path = shortcut.Targetpath
    return target_path

# 해당 파일 타입에 대한 기본 프로그램 반환
def get_associated_program(ext):
    prog_id = ''
    try:
        with winreg.OpenKey(winreg.HKEY_CLASSES_ROOT, ext, 0, winreg.KEY_READ) as key:
            prog_id = winreg.QueryValueEx(key, None)[0]
        with winreg.OpenKey(winreg.HKEY_CLASSES_ROOT, f'{prog_id}\\shell\\open\\command', 0, winreg.KEY_READ) as key:
            command = winreg.QueryValueEx(key, None)[0]
            
        # 필요한 명령 부분만 추출하도록 수정
        match = re.search(r'"(.+?\.exe)"', command)
        if match:
            command = match.group(1)
        else:
            command = command.split()[0]
        
        return os.path.basename(command)
    except FileNotFoundError:
        return f'{ext} 파일에 대한 기본 프로그램을 찾을 수 없습니다.'

# 2주내로 사용한 파일의 정보 정렬, 출력, 저장"
def sort_two_weeks():
    folder_path = os.path.join(os.getenv('APPDATA'), 'Microsoft', 'Windows', 'Recent')
    file_list = os.listdir(folder_path)
    today = date.today()
    two_weeks_ago = today - timedelta(days=14)
    # 데이터 저장 및 출력을 위한 리스트
    data = []
    
    # 파일 목록을 수정 시간 기준으로 정렬 (내림차순)
    sorted_files = sorted(file_list, key=lambda x: os.path.getmtime(os.path.join(folder_path, x)), reverse=True)
    
    # 2주간 사용한 파일 목 딕셔너리 형태로 저장
    recent_files = {}
    for f_name in sorted_files:
        file_path = os.path.join(folder_path, f_name)
        time = os.path.getmtime(file_path)
        file_date = datetime.fromtimestamp(time).date()
        if two_weeks_ago <= file_date <= today:
            recent_files[f_name] = {'file_date': file_date, 'program': ''}
    
    # 어떤 프로그램으로 열렸는지 확인
    for f_name, file_info in recent_files.items():
        file_ext = os.path.splitext(f_name)[1]
        if file_ext == '.lnk': # 바로가기 파일인 경우 처리
            file_path = os.path.join(folder_path, f_name)
            target_path = get_target_path(file_path)
            if target_path:
                file_ext = os.path.splitext(target_path)[1]
                file_info['program'] = get_associated_program(file_ext)
            else:
                file_info['program'] = '바로가기 대상 파일을 찾을 수 없습니다.'
        else:
            file_info['program'] = get_associated_program(file_ext)
    
    #딕셔너리 형태로 저장된 최근 2주간 사용한 파일 목록을 출력. 데이터셋 저장
    for f_name, file_info in recent_files.items():
        skip_file = False

        base_name, ext = os.path.splitext(f_name)
        if ext == ".lnk":
            base_name, ext = os.path.splitext(
                os.path.basename(get_target_path(os.path.join(folder_path, f_name)))
            )

        program = os.path.basename(file_info["program"])
        
        # 기본 프로그램을 찾을 수 없거나 바로가기 대상 파일을 찾을 수 없는 경우 체크 후 출력하지 않음
        if "기본 프로그램을 찾을 수 없습니다" in program or "바로가기 대상 파일을 찾을 수 없습니다" in program:
            skip_file = True

        if not skip_file:
            data.append((base_name, ext, program))
    
    #사용자의 홈 디렉토리 경로 얻고, 전체 경로 얻기
    csv_path=os.path.join(os.path.expanduser("~"),"Project404")
    
    # 해당 경로에 폴더가 존재하지 않는 경우, 폴더 생성
    if not os.path.exists(csv_path):
        os.mkdir(csv_path)
        
    # data 리스트를 DataFrame으로 변환
    df = pd.DataFrame(data, columns=['name', 'ext', 'program'])
    
    # CSV 파일로 저장
    file_path = os.path.join(csv_path, 'data.csv')
    df.to_csv(file_path, index=False, encoding='CP949')


if __name__ == "__main__":
    sort_two_weeks()
